In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *
from scripts.sm_utils import *

In [2]:
config = load_config()
od = ''

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [20]:
meta_file = '../config.tsv'
df = parse_config(meta_file)
meta = load_meta()
meta = meta.loc[meta.merged_run_mode==True]
sample_d = dict([(entry.cell_line_id, entry['sample']) \
                 for ind, entry in meta.iterrows()])
df['sample'] = df['sample'].map(sample_d)
samples = df['sample'].unique().tolist()

In [23]:
df

,lab_rep,sample,tech_rep_num,tech_rep
0,10_NI5_GM19117,YRI5,1,GM19117_1
1,11_NI6_GM19129,YRI6,1,GM19129_1
2,12_NI7_GM19240,YRI7,1,GM19240_1
3,13_CH1_GM18542,HAC1,1,GM18542_1
4,14_CH2_GM18561,HAC2,1,GM18561_1
5,15_CH3_GM18631,HAC3,1,GM18631_1
6,16_CH4_GM18772,HAC4,1,GM18772_1
7,17_CH5_GM18774,HAC5,1,GM18774_1
8,18_CH6_HG00621,HAC6,1,HG00621_1
9,19_KE1_GM19307,LWK1,1,GM19307_1


In [22]:
samples

['YRI5',
 'YRI6',
 'YRI7',
 'HAC1',
 'HAC2',
 'HAC3',
 'HAC4',
 'HAC5',
 'HAC6',
 'LWK1',
 'MPC1',
 'LWK2',
 'LWK3',
 'LWK4',
 'LWK5',
 'AJI1',
 'AJI2',
 'AJI3',
 'AJI4',
 'AJI5',
 'PEL1',
 'MPC2',
 'PEL2',
 'PEL3',
 'PEL4',
 'PEL5',
 'PEL6',
 'ITU1',
 'ITU2',
 'ITU3',
 'ITU4',
 'ITU5',
 'MPC3',
 'AJI6',
 'CEU1',
 'CEU2',
 'CEU3',
 'CEU4',
 'CEU5',
 'MPC4',
 'YRI1',
 'YRI2',
 'YRI3']

In [8]:
sample_d

{'HG04217': 'ITU5',
 'HG04216': 'ITU4',
 'HG03732': 'ITU3',
 'HG03729': 'ITU2',
 'HG03719': 'ITU1',
 'HG02293': 'PEL6',
 'HG02261': 'PEL5',
 'HG01975': 'PEL4',
 'HG01952': 'PEL3',
 'HG01928': 'PEL2',
 'HG01567': 'PEL1',
 'HG00621': 'HAC6',
 'GM24385': 'AJI6',
 'GM22305': 'AJI5',
 'GM22304': 'AJI4',
 'GM22300': 'AJI3',
 'GM22299': 'AJI2',
 'GM22234': 'AJI1',
 'GM19446': 'LWK5',
 'GM19390': 'LWK4',
 'GM19352': 'LWK3',
 'GM19328': 'LWK2',
 'GM19307': 'LWK1',
 'GM19240': 'YRI7',
 'GM19129': 'YRI6',
 'GM19117': 'YRI5',
 'GM19093': 'YRI4',
 'GM18906': 'YRI3',
 'GM18774': 'HAC5',
 'GM18772': 'HAC4',
 'GM18631': 'HAC3',
 'GM18561': 'HAC2',
 'GM18542': 'HAC1',
 'GM18489': 'YRI2',
 'GM18486': 'YRI1',
 'GM12878': 'CEU5',
 'GM12829': 'CEU4',
 'GM12812': 'CEU3',
 'GM12778': 'CEU2',
 'GM12273': 'CEU1',
 'GM10496': 'MPC5',
 'GM10495': 'MPC4',
 'GM10494': 'MPC3',
 'GM10493': 'MPC2',
 'GM10492': 'MPC1'}